# 1: 기본 설정 및 환경 확인

In [1]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import wandb
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder

from pathlib import Path
from dotenv import load_dotenv

print("🔧 환경 설정 및 WandB 초기화")
print("=" * 60)

🔧 환경 설정 및 WandB 초기화


In [2]:
# ============================================================
# 1. .env 파일 직접 파싱 (load_dotenv 우회)
# ============================================================
current_dir = Path(os.getcwd())
project_root = current_dir.parent
env_path = project_root / '.env'

print(f"📂 프로젝트 루트: {project_root}")
print(f"📄 .env 경로: {env_path}")

# 환경변수 직접 로드
env_vars = {}
if env_path.exists():
    print("✅ .env 파일 발견 - BOM 제거 파싱 중...")
    try:
        # 핵심 해결책: utf-8-sig 인코딩으로 BOM 자동 제거
        with open(env_path, 'r', encoding='utf-8-sig') as f:
            for line_num, line in enumerate(f, 1):
                line = line.strip()
                
                # 빈 줄이나 주석 무시
                if not line or line.startswith('#'):
                    continue
                
                # KEY=VALUE 형식 파싱
                if '=' in line:
                    key, value = line.split('=', 1)
                    key = key.strip()
                    value = value.strip()
                    
                    # 추가 안전장치: 보이지 않는 문자들 제거
                    key = key.replace('\ufeff', '').replace('\u200b', '').replace('\u2060', '')
                    value = value.replace('\ufeff', '').replace('\u200b', '').replace('\u2060', '')
                    
                    # 따옴표 제거
                    if (value.startswith('"') and value.endswith('"')) or \
                       (value.startswith("'") and value.endswith("'")):
                        value = value[1:-1]
                    
                    # 환경변수에 설정
                    os.environ[key] = value
                    env_vars[key] = value
                    
                    # 로그 출력 (이제 정상적으로 마스킹됨)
                    if key == 'WANDB_API_KEY':
                        print(f"  ✅ {key}: {value[:8]}****** (길이: {len(value)}) - BOM 제거 성공!")
                    else:
                        print(f"  ✅ {key}: {value}")
                else:
                    print(f"  ⚠️ 라인 {line_num} 형식 오류: {line}")
                    
        print(f"\n📊 총 {len(env_vars)}개 환경변수 로드 완료")
        
    except Exception as e:
        print(f"❌ .env 파일 읽기 오류: {e}")
        import traceback
        traceback.print_exc()
else:
    print("❌ .env 파일을 찾을 수 없습니다.")

📂 프로젝트 루트: c:\Users\ico26\kaggle\Heart Disease
📄 .env 경로: c:\Users\ico26\kaggle\Heart Disease\.env
✅ .env 파일 발견 - BOM 제거 파싱 중...
  ✅ WANDB_API_KEY: wandb_v1****** (길이: 86) - BOM 제거 성공!
  ✅ WANDB_PROJECT: heart-disease-prediction
  ✅ WANDB_ENTITY: icoghkdqh742-

📊 총 3개 환경변수 로드 완료


In [5]:
# ============================================================
# 2. 환경변수 로드 및 검증
# ============================================================
WANDB_API_KEY = os.environ.get('WANDB_API_KEY')
WANDB_PROJECT = os.environ.get('WANDB_PROJECT', 'heart-disease-prediction')
WANDB_ENTITY = os.environ.get('WANDB_ENTITY', None)

print(f"\n🔐 최종 인증 정보:")
print(f"  📁 Project: {WANDB_PROJECT}")
print(f"  👤 Entity: {WANDB_ENTITY}")

# API Key 검증 및 대체 입력
if not WANDB_API_KEY or len(WANDB_API_KEY.strip()) == 0:
    print("  ❌ API Key: 없음 또는 빈 값")
    print("\n" + "="*50)
    print("💡 API Key 수동 입력")
    print("="*50)
    
    from getpass import getpass
    WANDB_API_KEY = getpass("WandB API Key를 입력하세요 (입력 내용 숨김): ").strip()
    
    if not WANDB_API_KEY:
        raise ValueError("API Key가 필요합니다.")
    
    print(f"✅ 수동 입력 완료: {WANDB_API_KEY[:8]}******")
else:
    print(f"  ✅ API Key: {WANDB_API_KEY[:8]}****** (길이: {len(WANDB_API_KEY)})")


🔐 최종 인증 정보:
  📁 Project: heart-disease-prediction
  👤 Entity: icoghkdqh742-
  ✅ API Key: wandb_v1****** (길이: 86)


In [3]:
# ============================================================
# 3. WandB 오프라인 모드 강제 해제
# ============================================================
print(f"\n🔧 WandB 설정 정리...")

# 기존 WandB 실행 종료
if wandb.run is not None:
    wandb.finish()
    print("  🧹 이전 WandB 실행 종료")

# 오프라인 모드 관련 환경변수 제거
problematic_vars = ['WANDB_MODE', 'WANDB_DISABLED', 'WANDB_SILENT']
for var in problematic_vars:
    if var in os.environ:
        print(f"  🗑️ 제거: {var}={os.environ[var]}")
        del os.environ[var]

# 강제 온라인 모드 설정
os.environ['WANDB_MODE'] = 'online'
print("  ✅ 온라인 모드 강제 설정")

# notebooks/wandb 폴더 확인 (오프라인 설정 캐시)
wandb_cache_dir = current_dir / 'wandb'
if wandb_cache_dir.exists():
    print(f"  ⚠️ WandB 캐시 폴더 발견: {wandb_cache_dir}")
    print("  💡 오프라인 모드 문제 시 이 폴더를 삭제하세요")


🔧 WandB 설정 정리...
  ✅ 온라인 모드 강제 설정


In [9]:
# ============================================================
# 4. WandB 로그인 및 초기화
# ============================================================
try:
    print(f"\n🔐 WandB 로그인 시도...")
    wandb.login(key=WANDB_API_KEY, relogin=True)
    print("✅ WandB 로그인 성공!")
    
except Exception as e:
    print(f"❌ 로그인 실패: {e}")
    print("💡 계속 진행하여 init에서 재시도합니다...")

# 실험 설정
config = {
    'project_name': WANDB_PROJECT,
    'model_type': 'tabular',
    'batch_size': 64,
    'learning_rate': 0.001,
    'epochs': 100,
    'optimizer': 'Adam',
    'loss_function': 'BCEWithLogitsLoss',
    'hidden_dims': [256, 128, 64, 32],
    'dropout_rate': 0.3,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
}

# WandB 초기화
print(f"\n🚀 WandB 실험 초기화...")
try:
    run = wandb.init(
        project=WANDB_PROJECT,
        entity=WANDB_ENTITY,
        config=config,
        name=f"heart_disease_exp_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}",
        notes="Heart disease prediction using tabular neural network",
        tags=["pytorch", "tabular", "classification", "heart-disease"],
        save_code=True,
        mode="online",  # 명시적 온라인 모드
        reinit=True,
        settings=wandb.Settings(
            start_method="thread",  # Windows 호환성
            _disable_stats=False,
        )
    )
    print("✅ WandB 초기화 성공!")
    
except Exception as e:
    print(f"❌ WandB 초기화 실패: {e}")
    print(f"에러 타입: {type(e).__name__}")
    
    # 상세 에러 출력
    import traceback
    print("\n📋 상세 에러 정보:")
    traceback.print_exc()
    
    # 대안 제시
    print(f"\n💡 해결 방법:")
    print(f"1. notebooks/wandb 폴더 삭제 후 커널 재시작")
    print(f"2. 터미널에서 'wandb login --relogin' 실행")
    print(f"3. VPN/방화벽 설정 확인")
    
    raise

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.



🔐 WandB 로그인 시도...
✅ WandB 로그인 성공!

🚀 WandB 실험 초기화...


✅ WandB 초기화 성공!


In [7]:
# ============================================================
# 5. 기본 환경 설정
# ============================================================
# GPU/CPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n🖥️ 사용 디바이스: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

# 시드 고정
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# 출력 디렉토리 생성
os.makedirs(project_root / 'outputs' / 'models', exist_ok=True)
os.makedirs(project_root / 'outputs' / 'submissions', exist_ok=True)



🖥️ 사용 디바이스: cuda
   GPU: NVIDIA GeForce RTX 4060 Ti


In [11]:
# ============================================================
# 6. 최종 결과 출력
# ============================================================
print("\n" + "=" * 80)
print("🎉 WandB 실험 추적 시작!")
print("=" * 80)
print(f"프로젝트: {WANDB_PROJECT}")
print(f"실험명: {run.name}")
print(f"실험 ID: {run.id}")
print(f"Entity: {WANDB_ENTITY}")

# URL 확인 및 출력
import time
print(f"\n⏳ 실험 URL 생성 대기...")
max_wait = 5
for i in range(max_wait):
    if run.url:
        break
    time.sleep(1)

if run.url:
    print(f"✅ 실험 추적 URL: {run.url}")
    print(f"\n🔗 브라우저에서 실시간 추적: {run.url}")
else:
    # 수동 URL 생성
    manual_url = f"https://wandb.ai/{WANDB_ENTITY}/{WANDB_PROJECT}/runs/{run.id}"
    print(f"📍 실험 URL (수동): {manual_url}")
    print(f"\n🔗 브라우저에서 확인: {manual_url}")

print("=" * 80)
print("✅ 모든 설정 완료! 이제 데이터 로드를 진행하세요.")

# WandB 모델 모니터링 준비
print(f"\n📊 실험 추적 준비 완료")


🎉 WandB 실험 추적 시작!
프로젝트: heart-disease-prediction
실험명: heart_disease_exp_20260213_130457
실험 ID: ztv9a8fs
Entity: icoghkdqh742-

⏳ 실험 URL 생성 대기...
✅ 실험 추적 URL: https://wandb.ai/icoghkdqh742-/heart-disease-prediction/runs/ztv9a8fs

🔗 브라우저에서 실시간 추적: https://wandb.ai/icoghkdqh742-/heart-disease-prediction/runs/ztv9a8fs
✅ 모든 설정 완료! 이제 데이터 로드를 진행하세요.

📊 실험 추적 준비 완료


# 데이터 로드 및 ERD